In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
%matplotlib inline

"""トレーニングデータの読み込み"""
# 長さ128(2.56sec)のトレーニングデータが7521件/21人分
acc_x_train_df=pd.read_csv('./UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt', header=None, sep='\s+', na_values='na')
acc_y_train_df=pd.read_csv('./UCI HAR Dataset/train/Inertial Signals/total_acc_y_train.txt', header=None, sep='\s+', na_values='na')
acc_z_train_df=pd.read_csv('./UCI HAR Dataset/train/Inertial Signals/total_acc_z_train.txt', header=None, sep='\s+', na_values='na')

# Dataframeからarrayへ変換
acc_x_train=acc_x_train_df.values
acc_y_train=acc_y_train_df.values
acc_z_train=acc_z_train_df.values

# 3軸を横に並べる
X_train=np.c_[acc_x_train, acc_y_train, acc_z_train]

# 正解ラベル
    # 1 WALKING
    # 2 WALKING_UPSTAIRS
    # 3 WALKING_DOWNSTAIRS
    # 4 SITTING
    # 5 STANDING
    # 6 LAYING
train_label_df=pd.read_csv('./UCI HAR Dataset/train/Inertial Signals/y_train.txt', header=None, sep='\s+', na_values='na')
y_train=train_label_df.values.reshape(-1)
le = LabelEncoder()
y_train = le.fit_transform(y_train)

# 正解ラベルをone hot表現にする
#ohe = OneHotEncoder()
#y_train = ohe.fit_transform(train_label).A 

"""トレーニングデータの読み込み"""
# 長さ128(2.56sec)のトレーニングデータが7521件/21人分
acc_x_test_df=pd.read_csv('./UCI HAR Dataset/test/Inertial Signals/total_acc_x_test.txt', header=None, sep='\s+', na_values='na')
acc_y_test_df=pd.read_csv('./UCI HAR Dataset/test/Inertial Signals/total_acc_y_test.txt', header=None, sep='\s+', na_values='na')
acc_z_test_df=pd.read_csv('./UCI HAR Dataset/test/Inertial Signals/total_acc_z_test.txt', header=None, sep='\s+', na_values='na')

# Dataframeからarrayへ変換
acc_x_test=acc_x_test_df.values
acc_y_test=acc_y_test_df.values
acc_z_test=acc_z_test_df.values

# 3軸を横に並べる
X_test=np.c_[acc_x_test, acc_y_test, acc_z_test]

# 正解ラベル
    # 1 WALKING
    # 2 WALKING_UPSTAIRS
    # 3 WALKING_DOWNSTAIRS
    # 4 SITTING
    # 5 STANDING
    # 6 LAYING
test_label_df=pd.read_csv('./UCI HAR Dataset/test/Inertial Signals/y_test.txt', header=None, sep='\s+', na_values='na')
y_test=test_label_df.values.reshape(-1)
le = LabelEncoder()
y_test = le.fit_transform(y_test)

In [2]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers, 
                            batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x, h0=None, l=None):
        x, h = self.lstm(x, h0)
        # 最後のステップのみ取り出す
        x = x[:, -1, :]
        # 取り出した最後のステップを線形層に入れる
        x = self.linear(x)
        # 余分な次元を削除する
        # (batch_size, 1) -> (batch_size, )
        x = x.squeeze()
        return x

# 学習用関数
def train(loader_train, model_obj, optimizer, loss_fn, device, total_epoch, epoch):
    
    model_obj.train() # モデルを学習モードに変更
 
    # ミニバッチごとに学習
    for data, targets in loader_train:
 
        data = data.to(device) # GPUを使用するため，to()で明示的に指定
        targets = targets.to(device) # 同上
 
        optimizer.zero_grad() # 勾配を初期化
        outputs = model_obj(data) # 順伝播の計算
        loss = loss_fn(outputs, targets) # 誤差を計算
        loss.backward() # 誤差を逆伝播させる
        optimizer.step() # 重みを更新する
    
    print ('Epoch [%d/%d], Loss: %.4f' % (epoch, total_epoch, loss.item()))
 
 
# テスト用関数
def test(loader_test, trained_model, device):
 
    trained_model.eval() # モデルを推論モードに変更
    correct = 0 # 正解率計算用の変数を宣言
 
    # ミニバッチごとに推論
    with torch.no_grad(): # 推論時には勾配は不要
        for data, targets in loader_test:
 
            data = data.to(device) #  GPUを使用するため，to()で明示的に指定
            targets = targets.to(device) # 同上
 
            outputs = trained_model(data) # 順伝播の計算
 
            # 推論結果の取得と正誤判定
            _, predicted = torch.max(outputs.data, 1) # 確率が最大のラベルを取得
            correct += predicted.eq(targets.data.view_as(predicted)).sum() # 正解ならば正解数をカウントアップ
    
    # 正解率を計算
    data_num = len(loader_test.dataset) # テストデータの総数
    print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

In [ ]:
# 1. GPUの設定（PyTorchでは明示的に指定する必要がある）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# 2. ハイパーパラメータの設定（最低限の設定）
batch_size = len(y_train)
num_classes = 6
epochs = 500

# 5-2. データのフォーマットを変換：PyTorchでの形式 = [batch size，seq len，input size]
X_train = X_train.reshape(-1, 3, 128).transpose((0,2,1))
X_test = X_test.reshape(-1, 3, 128).transpose((0,2,1))

# 5-3. PyTorchのテンソルに変換
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)
 
# 5-4. 入力(x)とラベル(y)を組み合わせて最終的なデータを作成
ds_train = TensorDataset(X_train, y_train)
ds_test = TensorDataset(X_test, y_test)
 
# 5-5. DataLoaderを作成
loader_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(ds_test, batch_size=batch_size, shuffle=False)
 
# 6. モデル作成
model = LSTM(input_size=3, 
             hidden_size=64, 
             output_size=num_classes, 
             num_layers=2)
model.to(device)
print(model) # ネットワークの詳細を確認用に表示
 
# 7. 損失関数を定義
loss_fn = nn.CrossEntropyLoss()
 
# 8. 最適化手法を定義（ここでは例としてAdamを選択）
optimizer = optim.Adam(model.parameters(), lr=0.01)
 
# 9. 学習（エポック終了時点ごとにテスト用データで評価）
print('Begin train')
for epoch in range(1, epochs+1):
    train(loader_train, model, optimizer, loss_fn, device, epochs, epoch)
    test(loader_test, model, device)

cpu
LSTM(
  (lstm): LSTM(3, 64, num_layers=2, batch_first=True)
  (linear): Linear(in_features=64, out_features=6, bias=True)
)
Begin train
Epoch [1/500], Loss: 1.7940

Accuracy: 1000/2947 (33%)

Epoch [2/500], Loss: 1.7720

Accuracy: 1026/2947 (34%)

Epoch [3/500], Loss: 1.7318

Accuracy: 1026/2947 (34%)

Epoch [4/500], Loss: 1.5675

Accuracy: 1026/2947 (34%)

Epoch [5/500], Loss: 1.4210

Accuracy: 1026/2947 (34%)

Epoch [6/500], Loss: 1.4278

Accuracy: 990/2947 (33%)

Epoch [7/500], Loss: 1.3849

Accuracy: 1006/2947 (34%)

Epoch [8/500], Loss: 1.3765

Accuracy: 959/2947 (32%)

Epoch [9/500], Loss: 1.3739

Accuracy: 959/2947 (32%)

Epoch [10/500], Loss: 1.3682

Accuracy: 959/2947 (32%)

Epoch [11/500], Loss: 1.3634

Accuracy: 1026/2947 (34%)

Epoch [12/500], Loss: 1.3582

Accuracy: 1000/2947 (33%)

Epoch [13/500], Loss: 1.3590

Accuracy: 1000/2947 (33%)

Epoch [14/500], Loss: 1.3555

Accuracy: 1000/2947 (33%)

Epoch [15/500], Loss: 1.3505

Accuracy: 1115/2947 (37%)

Epoch [16/500], Lo